In [2]:
import matplotlib.pyplot as plt
import plotly.express as px
from tqdm import tqdm
import pandas as pd
import time
import csv
import os

## Funzione per trasformare i dati in CSV
Lo script prende in input un file .txt con il parametro 'rb' quindi in lettura in formato bytecode che vuol dire che dovrà essere decodificato.
Per prima cosa si scorre fino alla riga che inizia con 'NR' perché i dati iniziano da li, quando la trovo imposto flag_data a 1 che serve a dirmi che da li in poi dovrò salvare le righe.
Cerco la prima riga che inizia con '0' e la salto settando firsr_row a 1.
Quando sia data_flag che first_row sono a 1 allora inizio a salvare le righe.

Per salvare le righe:
* prima le decodifico con .decode()
* poi le divido con .split() che se usato senza parametri splitta la stringa in base agli spazi
* unisco i valori delle ultime due colonne perché fanno entrambi riferimento alla data
* elimino l'ultima riga perché l'ho unita alla penultima
* faccio un append della riga


In [4]:
def txt_to_csv(filename, filedest):
    
    
    if os.path.isfile(filedest + '.csv'):
        os.remove(filedest + '.csv')
        
    file1 = open(filename + '.txt', "rb")
    file_csv = open(filedest + '.csv', 'w+', newline ='')

    count = 0
    flag_data = 0

    with file_csv:
        for line in file1:

            if b'\xb0C' in line:     # se trovo il carattere °C lo ignoro
                pass
            else:
                line = line.decode() # altrimenti decodifico la riga

                # cerco la riga con i nomi della tabella
                if line.startswith('NR'):
                    flag_data = 1                  # quando trovo i nomi delle colonne imposto flag_data = 1
                    table_names = line.split()     # divido i nomi delle colonne      
                    table_names.remove('LastLine') # elimino 'LastLine' che fa riferimento ad un singolo dato duplicato
                    #for item in table_names: print(item)
                    write = csv.writer(file_csv)  
                    write.writerow(table_names)

                # Se sono arrivato alla tabella ma la riga inizia con 0 la salto
                elif flag_data == 1 and line.startswith('0'):
                    pass

                # Se sono arrivato alla tabella (flag_data == 1) e non sono alla prima riga
                # inizio a salvare i dati
                elif flag_data == 1:
                    row = line.split()

                    if not line.startswith('[END]') and len(row) == len(table_names)+1: # le righe con meno di 25 elementi hanno dati mancanti
                        row[-2] = str(row[-2]) + ' ' + str(row[-1]) # unisco l'ora col giorno
                        del row[-1] # elimino la colonna con i giorni
                        write = csv.writer(file_csv)
                        write.writerow(row)
                        count = count+1      # aggiorno il contatore delle righe

    #print(str(count) + ' righe salvate.')

#txt_to_csv('Tabelle_omv/TabLogAntTHS')

Per convertire tutti i file da txt a csv mi basta iterare all'interno della cartella e passare ogni file alla funzione txt_to_csv che prende in input il nome del file da convertire e il percorso di destinazione del file.

In [ ]:
import os

for filename in os.listdir('Tabelle_omv/'):
    if filename.endswith(".txt"): 
        print(filename[:-4])
        txt_to_csv(filename = 'Tabelle_omv/' + filename[:-4],
                   filedest = 'Tabelle_omv/csv/' + filename[:-4])
        print('\n')
        

## Funzione per caricare il CSV in un DataFrame
La funzione prende in input il nome del file .csv da caricare, converte la colonna DataTime in datetime e la imposta come indice del dataframe. Controlla se il dataframe è ordinato e se non lo è lo riordina. Elimina la colonna NR. Restituisce il dataframe.

In [6]:
def load_data(file):
    #column_subset = ['t_Mach',   't_HydrTa', 't_TTlubr', 't_TT_ab1', 
                     #'t_TT_ab2', 't_HdStUp', 't_HdStLw', 't_Sfrnt1', 
                     #'t_Sfrnt2', 't_Srear',  't_Smotor', 'DataTime']

    df = pd.read_csv (file,
                      #usecols=column_subset,
                      index_col=False)

    df['DataTime'] = pd.to_datetime(df['DataTime'], format='%H:%M:%S %d.%m.%Y')
    
    df.set_index('DataTime', drop = True, inplace=True)
    
    if not df.index.is_monotonic:
        df.sort_index(inplace = True)
    
    df.drop('NR', axis=1, inplace=True)
    
    return df

In [ ]:
import os

for filename in os.listdir('Tabelle_omv/csv/'):
    if filename.endswith(".csv"): 
        print(filename)
        data = load_data('Tabelle_omv/csv/' + filename)
        data.info()
        print('\n')

In [ ]:
path = 'Tabelle_omv/csv/'

data1 = load_data(path + 'TabLogAntTHS.csv')
data2 = load_data(path + 'TabLogAntTHS2.csv')
data3 = load_data(path + 'TabLogAntTHS3.csv')
data4 = load_data(path + 'TabLogAntTHS4.csv')

data_all = pd.concat([data1, data2, data3, data4])
print(data_all.index.is_monotonic)
data_all.sort_index(inplace = True)
print(data_all.index.is_monotonic)

data_all.info()
print(pd.date_range(start = '2020-02-24 17:55:01', end = '2020-05-06 02:55:01' ).difference(data_all.index))

## Processo per caricare tutti i file dello stesso tipo

In [22]:
path = 'Tabelle_omv/'
data_file = 'TabLogPostTHS'
data_list=[]

for filename in os.listdir(path):
    if filename.endswith(".txt") and data_file in filename: 
        print(filename[:-4])
        txt_to_csv(filename = path + filename[:-4],
                   filedest = path + 'csv/' + filename[:-4])
        
        data = load_data(path + 'csv/' + filename[:-4] + '.csv')
        data_list.append(data)
        
data_all = pd.concat(data_list)
data_all.sort_index(inplace = True)

print(pd.date_range(start = '2020-02-21 22:50:01', end = '2020-04-02 08:52:01' ).difference(data_all.index))

#data_all = data_all.loc['2020-04-02 01:05:00':'2020-05-04 23:05:00']
#data_all.drop('OpMode', axis=1, inplace=True)
#data_all = data_all.resample('1T').mean()
#data_all = data_all.round(2)

data_all.info()
#print(pd.date_range(start = '2020-02-21 22:00:00', end = '2020-04-02 08:00:00' ).difference(data_all.index))
#data_all.loc['2020-03-05 11:00:00':'2020-03-05 14:00:00']
#fig = px.line(data_all)
#fig.show()

data_all.to_csv(data_file + '_all')

TabLogPostTHS3
TabLogPostTHS4
TabLogPostTHS
TabLogPostTHS2
DatetimeIndex([], dtype='datetime64[ns]', freq=None)


### Funzione che carica tutte le tabelle dello stesso tipo e le riunisce nello stesso df
Per velocizzare il caricamento dei dati unisco tutte le tabelle in un unico file così devo caricare solo quello

In [23]:
def create_single_csv(path_folder, tab_name):
    data_list=[]

    for filename in os.listdir(path):
        if filename.endswith(".txt") and data_file in filename: 
            print(filename[:-4])
            txt_to_csv(filename = path + filename[:-4],
                       filedest = path + 'csv/' + filename[:-4])

            data = load_data(path + 'csv/' + filename[:-4] + '.csv')
            data_list.append(data)

    data_all = pd.concat(data_list)
    data_all.sort_index(inplace = True)
    data_all.to_csv(data_file + '_all')

#------- MAIN
path = 'Tabelle_omv/'
data_file = 'TabLogPostTHS'

create_single_csv(path, data_file)

TabLogPostTHS3
TabLogPostTHS4
TabLogPostTHS
TabLogPostTHS2
DatetimeIndex([], dtype='datetime64[ns]', freq=None)
